# Pandas 심화문제

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
pd.__version__

'1.0.5'

## 1. Iris - sns.load_dataset('iris')

In [3]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등 기초통계량(describe())을 구하시오.

In [4]:
iris.groupby('species').describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

b. 3분위수(Q3)와 1분위수(Q1)의 차이보다 1.5배가 크거나 작은 데이터는 이상치이다. 즉, 

```
Q1 - 1.5 * (Q3 - Q1) 보다 작은 데이터
Q3 + 1.5 * (Q3 - Q1) 보다 큰 데이터
```

이 이상치를 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [5]:
grps = iris.groupby('species')
q1 = grps.quantile(.25)
q3 = grps.quantile(.75)

iqr = q3 - q1
qs = q1 - 1.5 * iqr
ql = q3 + 1.5 * iqr

data = []
for i, grp in grps:
    tmp = iris.query(f"""
    (species in '{i}') \
        & ({qs['sepal_length'][i]} < sepal_length < {ql['sepal_length'][i]}) \
        & ({qs['sepal_width'][i]} < sepal_width < {ql['sepal_width'][i]}) \
        & ({qs['petal_length'][i]} < petal_length < {ql['petal_length'][i]}) \
        & ({qs['petal_width'][i]} < petal_width < {ql['petal_width'][i]})
    """)
    data.append(tmp)

pd.concat(data).groupby('species').agg(['mean', 'std'])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa         5.028571  0.335163    3.430952  0.329440     1.454762   
versicolor     5.953061  0.507076    2.775510  0.314597     4.285714   
virginica      6.584783  0.550946    2.965217  0.254049     5.543478   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.125333    0.228571  0.080504  
versicolor  0.437798    1.330612  0.197066  
virginica   0.504932    2.041304  0.269630

In [6]:
iris.groupby('species').agg(['mean', 'std'])

sepal_length           sepal_width           petal_length  \
                   mean       std        mean       std         mean   
species                                                                
setosa            5.006  0.352490       3.428  0.379064        1.462   
versicolor        5.936  0.516171       2.770  0.313798        4.260   
virginica         6.588  0.635880       2.974  0.322497        5.552   

                     petal_width            
                 std        mean       std  
species                                     
setosa      0.173664       0.246  0.105386  
versicolor  0.469911       1.326  0.197753  
virginica   0.551895       2.026  0.274650

## 2. Titanic - sns.load_dataset('titanic')

In [7]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True


a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. category1 카테고리는 다음과 같이 정의됨

1. 20살이 넘으면 성별을 그대로 사용한다.
2. 20살 미만이면 성별에 관계없이 "child" 라고 한다.

In [8]:
titanic['category1'] = titanic.apply(lambda x: x.sex if x.age > 20 else 'child', axis=1)
titanic[['age', 'category1']].tail(10)

,age,category1
881,33.0,male
882,22.0,female
883,28.0,male
884,25.0,male
885,39.0,female
886,27.0,male
887,19.0,child
888,NaN,child
889,26.0,male
890,32.0,male


b. 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터프레임을 고치시오.

In [9]:
titanic['age'] = titanic['age'].fillna(titanic['age'].mean())
titanic[['age']].tail()

,age
886,27.000000
887,19.000000
888,29.699118
889,26.000000
890,32.000000


c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

In [10]:
titanic = sns.load_dataset('titanic')

In [11]:
titanic[['survived', 'sex']].groupby('sex').mean()

,survived
sex,
female,0.742038
male,0.188908


In [12]:
titanic[['survived', 'class']].groupby('class').mean()

,survived
class,
First,0.629630
Second,0.472826
Third,0.242363


In [13]:
titanic[['survived', 'embark_town']].groupby('embark_town').mean()

,survived
embark_town,
Cherbourg,0.553571
Queenstown,0.389610
Southampton,0.336957


d. 타이타닉호 승객을 '미성년자', '청년', '중년', '장년', '노년' 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.

```
bins = [1, 20, 30, 40, 50, 70, 100]
labels = ['미성년자', '청년', '중년', '장년', '노년']
```

In [14]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanic['age_structure'] = pd.cut(titanic['age'], bins, labels=labels)
titanic[['age_structure', 'survived']].groupby('age_structure').mean()

,survived
age_structure,
미성년자,0.424242
청년,0.365217
중년,0.423237
장년,0.355932
노년,0.200000


e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [15]:
titanic['male'] = titanic.apply(lambda x: 1 if x.sex == 'male' else 0, axis=1)
titanic['female'] = titanic.apply(lambda x: 1 if x.sex == 'female' else 0, axis=1)
titanic['age_group'] = pd.qcut(titanic['age'], 3, labels=['young', 'median', 'old'])
titanic[['age_group', 'survived', 'male', 'female']].groupby('age_group').mean()

,survived,male,female
age_group,,,
young,0.418699,0.589431,0.410569
median,0.400862,0.659483,0.340517
old,0.398305,0.656780,0.343220


## 3. Mile Per Gallon - sns.load_dataset('mpg')

In [16]:
import seaborn as sns
mpg = sns.load_dataset('mpg')
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

In [17]:
mpg['hp_per_cc'] = mpg['horsepower'] / mpg['displacement']
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,0.423453
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,0.471429
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,0.471698
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,0.493421
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,0.463576


b. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

In [18]:
mpg['manufacturer'] = mpg['name'].str.split(' ', n=1, expand=True)[0]
mpg['model'] = mpg['name'].str.split(' ', n=1, expand=True)[1]
mpg.drop('name', axis='columns', inplace=True)
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,0.423453,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,0.471429,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,0.471698,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,0.493421,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,0.463576,ford,torino


c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

In [19]:
mpg[['cylinders', 'mpg']].groupby('cylinders').agg({'mpg':'mean'})

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

In [20]:
mpg[['origin', 'hp_per_cc']].groupby('origin').mean()

,hp_per_cc
origin,
europe,0.740005
japan,0.789485
usa,0.512063


e. 모델이 5개 이상인 제조사에 대하여 연비(mpg)의 평균이 가장 좋은 제조사 Top 5를 구하시오.

In [21]:
mpg[['manufacturer', 'model', 'mpg']].groupby('manufacturer').agg({'model': 'count', 'mpg': 'mean'}).query('model >= 5').sort_values(by='mpg', ascending=False).head(5)

,model,mpg
manufacturer,,
vw,6,39.016667
honda,13,33.761538
renault,5,32.880000
datsun,23,31.113043
mazda,10,30.860000
